In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
import zipfile

In [3]:
pwd

'/home/hadoop'

In [4]:
!ls

'${system:java.io.tmpdir}'      hadoop_test1
 2024.06.17.spark2.ipynb        hflight.csv
 2024.06.17.spark.ipynb         hive
 2024.06.18.spark.ipynb         hive3
 2024.06.19.spark2.ipynb        miniconda3
 2024.06.19.spark.ipynb         Miniconda3-py310_24.4.0-0-Linux-x86_64.sh
 2024-06-20-spark.ipynb         nohup.out
 2024-06-24-1.ipynb	        retail_data
 apache-hive-3.1.3-bin.tar.gz   retail-data-20240620T002036Z-001.zip
 apache-hive-4.0.0-bin.tar.gz   solo_study
 hadoop			        test.txt
 hadoop_data		        Untitled1.ipynb
 hadoop.tar.gz


In [5]:
files = zipfile.ZipFile("./retail-data-20240620T002036Z-001.zip")
files.extractall(path="./retail_data")

In [6]:
df = spark.read.format("csv").\
    option("header", "true").\
    option("inferSchema", "true").\
    load("file:///home/hadoop/retail_data/retail-data/by-day/*")

In [7]:
df.count()

531694

In [8]:
df.limit(3).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [9]:
df.createOrReplaceTempView("dfTable")

In [10]:
from pyspark.sql.functions import lit,col

In [11]:
df.select(lit(5), lit("five") , lit(5.0)).limit(3).show()

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+



In [12]:
df.limit(1).show()

+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|       Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+



# InvoiceNo 가 536365인 row값에서 InvoiceNo,Description 만 출력

In [13]:
df.filter(col("InvoiceNo") ==536365).select("InvoiceNo","Description").show()

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536365|WHITE HANGING HEA...|
|   536365| WHITE METAL LANTERN|
|   536365|CREAM CUPID HEART...|
|   536365|KNITTED UNION FLA...|
|   536365|RED WOOLLY HOTTIE...|
|   536365|SET 7 BABUSHKA NE...|
|   536365|GLASS STAR FROSTE...|
+---------+--------------------+



In [14]:
from pyspark.sql.functions import instr

filter_ = col("UnitPrice") > 600
filter_2 = instr(df.Description, "POSTAGE") >= 1


In [15]:
df.limit(5).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [16]:
df.where(df.StockCode.isin("DOT")).\
    where(filter_ | filter_2).limit(1).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   580610|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:48:00|  2196.67|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [17]:
sql="""select * from dfTable where UnitPrice > 600 or instr(Description,"POSTAGE") >=1 LIMIT 1
"""

In [18]:
spark.sql(sql).show()

+---------+---------+-----------+--------+-------------------+---------+----------+--------+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+-----------+--------+-------------------+---------+----------+--------+
|   580545|     POST|    POSTAGE|       1|2011-12-05 09:16:00|     28.0|   12782.0|Portugal|
+---------+---------+-----------+--------+-------------------+---------+----------+--------+



In [19]:
sql="""select * from dfTable where StockCode in ("DOT") AND (UnitPrice > 600 or instr(Description,"POSTAGE") >=1) LIMIT 3
"""

In [20]:
spark.sql(sql).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   580610|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:48:00|  2196.67|      null|United Kingdom|
|   580612|      DOT|DOTCOM POSTAGE|       1|2011-12-05 11:58:00|   2114.0|      null|United Kingdom|
|   580727|      DOT|DOTCOM POSTAGE|       1|2011-12-05 17:17:00|  1599.26|   14096.0|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [21]:
filter_1 = col("StockCode") == "DOT"
filter_2 = col("UnitPrice") >600
filter_3 = instr(col("Description") , "POSTAGE") >=1

In [22]:
# df.withColumn("isExpensive",filter_1 & (filter_2 | filter_3)).show()

In [23]:
# df.withColumn("isExpensive",filter_1 & (filter_2 | filter_3)).\
#     where("isExpensive").limit(5).show()

In [24]:
df.withColumn("isExpensive",filter_1 & (filter_2 | filter_3)).\
    where("isExpensive").limit(5).\
    select("unitPrice","isExpensive").limit(5).show()

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|  2196.67|       true|
|   2114.0|       true|
|  1599.26|       true|
|   1172.1|       true|
|   845.23|       true|
+---------+-----------+



In [25]:
# filter_1 = col("StockCode") == "DOT"
# filter_2 = col("UnitPrice") >600
# filter_3 = instr(col("Description") , "POSTAGE") >=1
sql="""select unitPrice,isExpensive from (select *,
CASE WHEN (StockCode = 'DOT' AND (UnitPrice > 600 OR INSTR(Description, 'POSTAGE') >= 1)) 
THEN 'True' 
ELSE 'False' 
END AS isExpensive
from dfTable) A 
where A.isExpensive ==True
limit 5
"""
spark.sql(sql).show()

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|  2196.67|       True|
|   2114.0|       True|
|  1599.26|       True|
|   1172.1|       True|
|   845.23|       True|
+---------+-----------+



In [26]:
encore = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://ip:port/encore_web") \
    .option("dbtable","encore_question") \
    .option("numPartitions",5) \
    .option("user", "user") \
    .option("password", 'password') \
    .load()

In [27]:
encore.printSchema()

root
 |-- id: long (nullable = true)
 |-- subject: string (nullable = true)
 |-- content: string (nullable = true)
 |-- create_date: timestamp (nullable = true)
 |-- quthor_id: integer (nullable = true)



In [28]:
encore.limit(5).show()

+---+-------+-------+--------------------+---------+
| id|subject|content|         create_date|quthor_id|
+---+-------+-------+--------------------+---------+
|  1|      d|      d|2024-05-14 06:37:...|        1|
|  2|      d|      d|2024-05-14 06:37:...|        1|
|  3|      d|      d|2024-05-14 06:37:...|        1|
|  4|sdfasdf| fasadf|2024-05-14 06:38:...|        1|
|  5|   우와|   우와|2024-05-14 06:38:...|        1|
+---+-------+-------+--------------------+---------+



In [29]:
from pyspark.sql.functions import instr

In [30]:
encore.where(instr(col("subject"),"점심") >=1).show()

+---+------------------------+-------+--------------------+---------+
| id|                 subject|content|         create_date|quthor_id|
+---+------------------------+-------+--------------------+---------+
| 77|내일 점심 뭐 드실건가요?| 배고픔|2024-05-16 06:20:...|        1|
+---+------------------------+-------+--------------------+---------+



In [31]:
# encore.filter(col("subject").like("%점심%")).limit(5).show()

In [32]:
from pyspark.sql.functions import expr, pow

In [33]:
pow(col("Quantity")*col("UnitPrice"),2)+5

Column<'(POWER((Quantity * UnitPrice), 2) + 5)'>

In [34]:
# (현재 수량 * 단가)^2 + 5 
condition1 = pow(col("Quantity") * col("UnitPrice"), 2) + 5 
df.select(col("CustomerId"), condition1.alias("realQuantity")   ).limit(3).show()

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   14075.0|         7387.2464|
|   14075.0|             630.0|
|   14075.0|1573.1599999999996|
+----------+------------------+



In [35]:
df.show(1)

+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|       Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



In [36]:
sql = """
select CustomerId, POWER((Quantity * UnitPrice), 2) + 5 as realQuantity
from dfTable
limit 10
"""

In [37]:
spark.sql(sql).show()

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   14075.0|         7387.2464|
|   14075.0|             630.0|
|   14075.0|1573.1599999999996|
|   14075.0|             905.0|
|   14075.0|239.08999999999997|
|   14075.0|1669.6399999999999|
|   14075.0|           1573.16|
|   14075.0|1650.1136000000001|
|   18180.0|             294.0|
|   18180.0|             294.0|
+----------+------------------+



In [38]:
from pyspark.sql.functions import round , bround, lit

In [39]:
from pyspark.sql.functions import round, bround, lit
df.select(round(lit("2.5")), bround(lit("2.5"))     ).show()


+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 20 rows



In [40]:
df.select(round(lit("2.4"))).show()

+-------------+
|round(2.4, 0)|
+-------------+
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
|          2.0|
+-------------+
only showing top 20 rows



In [41]:
from pyspark.sql.functions import corr

In [42]:
df.select(corr("Quantity","UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.00122463128643...|
+-------------------------+



In [43]:
df.describe().show()

24/06/20 03:29:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+
|  count|            531694|            531694|              530257|            531694|           531694|            401575|     531694|
|   mean| 560143.9877866305|27611.865304620373|             20713.0|  9.61681719184343|4.616183771117624| 15282.84867832908|       null|
| stddev|13382.920102875296|16779.170172522252|                null|219.96052512423964|97.64344281865462|1713.3126527231266|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06|           12346.0|  Australia|
|    max|           C581569|             

In [44]:
df.select(col("Quantity"),col("UnitPrice")).describe().show()

+-------+------------------+-----------------+
|summary|          Quantity|        UnitPrice|
+-------+------------------+-----------------+
|  count|            531694|           531694|
|   mean|  9.61681719184343|4.616183771117624|
| stddev|219.96052512423964|97.64344281865462|
|    min|            -80995|        -11062.06|
|    max|             80995|          38970.0|
+-------+------------------+-----------------+



In [45]:
from pyspark.sql.functions import count , mean, stddev_pop, min, max

In [46]:
df.select(mean("UnitPrice")).show()

+-----------------+
|   avg(UnitPrice)|
+-----------------+
|4.616183771117624|
+-----------------+



In [47]:
df.select(mean("UnitPrice"),stddev_pop("UnitPrice")).show()

+-----------------+---------------------+
|   avg(UnitPrice)|stddev_pop(UnitPrice)|
+-----------------+---------------------+
|4.616183771117624|    97.64335099564296|
+-----------------+---------------------+



In [48]:
df.stat.approxQuantile("UnitPrice",[0.5],0.05)

[2.08]

In [49]:
df.stat.approxQuantile("UnitPrice",[0.5],0)

[2.08]

In [50]:
from pyspark.sql.functions import initcap, col, lower, upper

In [51]:
df.select((col("Description"))).limit(2).show() 

+-------------------+
|        Description|
+-------------------+
| RABBIT NIGHT LIGHT|
|DOUGHNUT LIP GLOSS |
+-------------------+



In [52]:
df.select(initcap(col("Description"))).limit(2).show()

+--------------------+
|initcap(Description)|
+--------------------+
|  Rabbit Night Light|
| Doughnut Lip Gloss |
+--------------------+



In [53]:
df.select(col("Description"), lower(col("Description")),upper(col("Description"))).limit(3).show()

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|  RABBIT NIGHT LIGHT|  rabbit night light|  RABBIT NIGHT LIGHT|
| DOUGHNUT LIP GLOSS | doughnut lip gloss | DOUGHNUT LIP GLOSS |
|12 MESSAGE CARDS ...|12 message cards ...|12 MESSAGE CARDS ...|
+--------------------+--------------------+--------------------+



In [54]:
spark.sql("""select Description, lower(Description), upper(Description) from dfTable""").show()

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|  RABBIT NIGHT LIGHT|  rabbit night light|  RABBIT NIGHT LIGHT|
| DOUGHNUT LIP GLOSS | doughnut lip gloss | DOUGHNUT LIP GLOSS |
|12 MESSAGE CARDS ...|12 message cards ...|12 MESSAGE CARDS ...|
|BLUE HARMONICA IN...|blue harmonica in...|BLUE HARMONICA IN...|
|   GUMBALL COAT RACK|   gumball coat rack|   GUMBALL COAT RACK|
|SKULLS  WATER TRA...|skulls  water tra...|SKULLS  WATER TRA...|
|FELTCRAFT GIRL AM...|feltcraft girl am...|FELTCRAFT GIRL AM...|
|CAMOUFLAGE LED TORCH|camouflage led torch|CAMOUFLAGE LED TORCH|
|WHITE SKULL HOT W...|white skull hot w...|WHITE SKULL HOT W...|
|ENGLISH ROSE HOT ...|english rose hot ...|ENGLISH ROSE HOT ...|
|HOT WATER BOTTLE ...|hot water bottle ...|HOT WATER BOTTLE ...|
|SCOTTIE DOG HOT W...|scottie dog hot w...|SCOTTIE DOG HOT W...|
|ROSE CARAVAN DOOR...|ros

In [55]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

In [56]:
df.select(ltrim(lit("       Hi      ")),  rtrim(lit("       Hi    ")) ,
         trim(lit("       Hi      ")), 
         rpad(lit("hi"), 10, "-")).show()


+----------------------+--------------------+---------------------+---------------+
|ltrim(       Hi      )|rtrim(       Hi    )|trim(       Hi      )|rpad(hi, 10, -)|
+----------------------+--------------------+---------------------+---------------+
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--------|
|              Hi      |                  Hi|                   Hi|     hi--

In [57]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [58]:
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"

In [59]:
df.select(regexp_replace(col("Description"), regex_string,"COLOR").alias("color"),
              col("Description")).limit(10).show(truncate=False)

+-------------------------------+-------------------------------+
|color                          |Description                    |
+-------------------------------+-------------------------------+
|RABBIT NIGHT LIGHT             |RABBIT NIGHT LIGHT             |
|DOUGHNUT LIP GLOSS             |DOUGHNUT LIP GLOSS             |
|12 MESSAGE CARDS WITH ENVELOPES|12 MESSAGE CARDS WITH ENVELOPES|
|COLOR HARMONICA IN BOX         |BLUE HARMONICA IN BOX          |
|GUMBALL COAT RACK              |GUMBALL COAT RACK              |
|SKULLS  WATER TRANSFER TATTOOS |SKULLS  WATER TRANSFER TATTOOS |
|FELTCRAFT GIRL AMELIE KIT      |FELTCRAFT GIRL AMELIE KIT      |
|CAMOUFLAGE LED TORCH           |CAMOUFLAGE LED TORCH           |
|COLOR SKULL HOT WATER BOTTLE   |WHITE SKULL HOT WATER BOTTLE   |
|ENGLISH ROSE HOT WATER BOTTLE  |ENGLISH ROSE HOT WATER BOTTLE  |
+-------------------------------+-------------------------------+



In [60]:
spark.sql(""" select regexp_replace(Description, "BLACK|WHITE|RED|GREEN|BLUE", "color")
    as color, Description from dfTable

""").show(truncate=False)


+-----------------------------------+-----------------------------------+
|color                              |Description                        |
+-----------------------------------+-----------------------------------+
|RABBIT NIGHT LIGHT                 |RABBIT NIGHT LIGHT                 |
|DOUGHNUT LIP GLOSS                 |DOUGHNUT LIP GLOSS                 |
|12 MESSAGE CARDS WITH ENVELOPES    |12 MESSAGE CARDS WITH ENVELOPES    |
|color HARMONICA IN BOX             |BLUE HARMONICA IN BOX              |
|GUMBALL COAT RACK                  |GUMBALL COAT RACK                  |
|SKULLS  WATER TRANSFER TATTOOS     |SKULLS  WATER TRANSFER TATTOOS     |
|FELTCRAFT GIRL AMELIE KIT          |FELTCRAFT GIRL AMELIE KIT          |
|CAMOUFLAGE LED TORCH               |CAMOUFLAGE LED TORCH               |
|color SKULL HOT WATER BOTTLE       |WHITE SKULL HOT WATER BOTTLE       |
|ENGLISH ROSE HOT WATER BOTTLE      |ENGLISH ROSE HOT WATER BOTTLE      |
|HOT WATER BOTTLE KEEP CALM         |H

In [61]:
df.select( regexp_extract(col("Description"), regex_string, 0).alias("color"),
            col("Description")).limit(10).show(truncate=False)


+-----+-------------------------------+
|color|Description                    |
+-----+-------------------------------+
|     |RABBIT NIGHT LIGHT             |
|     |DOUGHNUT LIP GLOSS             |
|     |12 MESSAGE CARDS WITH ENVELOPES|
|BLUE |BLUE HARMONICA IN BOX          |
|     |GUMBALL COAT RACK              |
|     |SKULLS  WATER TRANSFER TATTOOS |
|     |FELTCRAFT GIRL AMELIE KIT      |
|     |CAMOUFLAGE LED TORCH           |
|WHITE|WHITE SKULL HOT WATER BOTTLE   |
|     |ENGLISH ROSE HOT WATER BOTTLE  |
+-----+-------------------------------+

